# Personalized Email Agent (Local)

This notebook creates a local email agent that:
- Generates personalized emails for multiple recipients
- Shows all emails before sending
- Sends via Gmail API only after your confirmation
- Runs completely locally on your device

## Step 1: Install Required Libraries
Use "requirements.txt" to install the required packages.

## Step 2: Import Libraries and Setup

In [ ]:
import os
import base64
import re
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import pandas as pd
from tabulate import tabulate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Gmail API scopes
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

print("✅ Libraries imported successfully!")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"💻 CUDA available: {torch.cuda.is_available()}")

## Step 3: Gmail Authentication Function

**First-time setup:**
1. Go to [Google Cloud Console](https://console.cloud.google.com/)
2. Create a new project or select existing one
3. Enable Gmail API
4. Create OAuth 2.0 credentials (Desktop app)
5. Download the credentials and save as `credentials.json` in the same folder as this notebook

In [ ]:
def authenticate_gmail():
    """Authenticate with Gmail API using OAuth2."""
    creds = None
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not os.path.exists('credentials.json'):
                print("❌ ERROR: credentials.json not found!")
                print("Please follow the setup instructions above to create OAuth credentials.")
                return None
            
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Save credentials for next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    print("✅ Gmail authentication successful!")
    return build('gmail', 'v1', credentials=creds)

## Step 4: Email Validation Function

In [ ]:
def validate_email(email):
    """Validate email format."""
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

## Step 5: Email Generation Function

## Step 5A: LLM Configuration (Qwen2.5 0.5B Instruct)

This agent uses **Qwen2.5 0.5B Instruct** from HuggingFace running locally on your machine.

**Model Details:**
- Model: `Qwen/Qwen2.5-0.5B-Instruct`
- Ultra-fast and lightweight model (0.5B parameters)
- Optimized for instruction following
- Works on CPU or GPU with FP16 precision
- First run will download the model (~1GB)
- **FASTEST option** - generates emails in seconds even on CPU!

The model will be loaded in the next cell.

In [ ]:
class QwenEmailGenerator:
    """Qwen2.5 0.5B Instruct model for email generation."""
    
    def __init__(self, model_name="Qwen/Qwen2.5-0.5B-Instruct"):
        self.model_name = model_name
        self.model = None
        self.tokenizer = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
    def load_model(self):
        """Load Qwen2.5 model with FP16 for efficient email generation."""
        if self.model is not None:
            print("✅ Model already loaded!")
            return True
        
        try:
            print(f"📥 Loading {self.model_name}...")
            print(f"💾 Device: {self.device}")
            print("⏳ First time will download ~1GB model. Please wait...")
            
            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )
            
            # Load model with FP16 precision for both GPU and CPU
            if torch.cuda.is_available():
                # GPU mode with FP16
                print("🔥 Loading with FP16 on GPU...")
                self.model = AutoModelForCausalLM.from_pretrained(
                    self.model_name,
                    device_map="auto",
                    trust_remote_code=True,
                    torch_dtype=torch.float16
                )
            else:
                # CPU mode with FP16
                print("💻 Loading with FP16 on CPU...")
                self.model = AutoModelForCausalLM.from_pretrained(
                    self.model_name,
                    trust_remote_code=True,
                    torch_dtype=torch.float16
                )
                self.model.to(self.device)
            
            print("✅ Qwen2.5 0.5B Instruct loaded successfully with FP16!")
            return True
            
        except Exception as e:
            print(f"❌ Failed to load model: {e}")
            return False
    
    def generate(self, prompt, max_length=1024, temperature=0.7):
        """Generate text using Qwen2.5."""
        if self.model is None:
            print("❌ Model not loaded. Call load_model() first.")
            return None
        
        try:
            # Format prompt for Qwen2.5 Instruct
            messages = [
                {"role": "system", "content": "You are a professional email writer."},
                {"role": "user", "content": prompt}
            ]
            
            # Apply chat template
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            
            # Tokenize
            inputs = self.tokenizer([text], return_tensors="pt").to(self.device)
            
            # Generate with FP16
            with torch.no_grad():
                with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):
                    outputs = self.model.generate(
                        **inputs,
                        max_new_tokens=max_length,
                        temperature=temperature,
                        do_sample=True,
                        top_p=0.9,
                        repetition_penalty=1.1,
                        pad_token_id=self.tokenizer.eos_token_id
                    )
            
            # Decode
            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Extract only the assistant's response
            if "assistant" in generated_text.lower():
                parts = generated_text.split("assistant")
                if len(parts) > 1:
                    generated_text = parts[-1].strip()
            
            # Remove any remaining chat template markers
            for marker in ["<|im_start|>", "<|im_end|>", "<|endoftext|>"]:
                generated_text = generated_text.replace(marker, "")
            
            return generated_text.strip()
            
        except Exception as e:
            print(f"❌ Generation failed: {e}")
            return None

# Initialize the generator
print("🚀 Initializing Qwen2.5 Email Generator...")
qwen_generator = QwenEmailGenerator()

# Load the model
qwen_generator.load_model()

In [ ]:
def generate_personalized_email_with_qwen(name, email, context):
    """Generate a truly personalized email using Qwen2.5 0.5B Instruct."""
    
    first_name = name.split()[0] if name else "there"
    
    # Create detailed prompt for Qwen2.5
    prompt = f"""Write a professional and personalized business email based on the following information:

RECIPIENT DETAILS:
- Full Name: {name}
- First Name: {first_name}
- Email Address: {email}

EMAIL PURPOSE AND CONTEXT:
{context}

INSTRUCTIONS:
1. Address the recipient by their first name ({first_name})
2. Write in a professional but warm and friendly tone
3. Make the email feel personal and genuine, not like a mass email
4. Keep it concise - 2 to 3 short paragraphs maximum
5. Include a relevant call-to-action based on the context
6. Be engaging and conversational

FORMAT YOUR RESPONSE EXACTLY AS:
SUBJECT: [Write a compelling subject line here]

BODY:
[Write the complete email body here]

Now generate the email:"""

    try:
        # Generate using Qwen2.5
        generated_text = qwen_generator.generate(prompt, max_length=4096, temperature=0.7)
        
        if not generated_text:
            raise Exception("No output from model")
        
        # Parse subject and body
        subject = ""
        body = ""
        
        lines = generated_text.strip().split('\n')
        current_section = None
        body_lines = []
        
        for line in lines:
            line = line.strip()
            if line.upper().startswith("SUBJECT:"):
                subject = line[8:].strip()  # Remove "SUBJECT:" prefix
                current_section = "subject"
            elif line.upper().startswith("BODY:"):
                current_section = "body"
            elif current_section == "body" and line:
                body_lines.append(line)
        
        # Fallback parsing if format not perfectly followed
        if not subject:
            # Try to find first non-empty line as subject
            for line in lines:
                cleaned = line.strip()
                if cleaned and not cleaned.upper().startswith("BODY:"):
                    subject = cleaned.replace("SUBJECT:", "").replace("Subject:", "").strip()
                    if len(subject) > 10:  # Valid subject line
                        break
        
        if not body_lines:
            # Use everything after "BODY:" or after first line as body
            found_body_marker = False
            skip_first = False
            
            for line in lines:
                if "BODY:" in line.upper():
                    found_body_marker = True
                    continue
                    
                if found_body_marker or skip_first:
                    cleaned = line.strip()
                    if cleaned and "SUBJECT:" not in cleaned.upper():
                        body_lines.append(cleaned)
                elif subject and subject in line:
                    skip_first = True
        
        # Join body lines
        body = '\n\n'.join([line for line in body_lines if line])
        
        # Clean up subject
        subject = subject.strip('"').strip("'").strip()
        subject = subject.replace("**", "").replace("*", "")  # Remove markdown
        
        # Validate and create fallback if needed
        if not subject or len(subject) < 5:
            subject = f"Important: {context[:40]}..." if len(context) > 40 else context
        
        if not body or len(body) < 30:
            raise Exception("Generated body too short or invalid")
        
        # Add signature
        body += f"\n\nBest regards,\nYour Name\n\n---\nThis email was sent to {email}"
        
        return subject, body
        
    except Exception as e:
        print(f"⚠️  Qwen2.5 generation failed for {name}: {e}")
        print("   Falling back to template...")
        return generate_personalized_email_template(name, email, context)


def generate_personalized_email_template(name, email, context):
    """Fallback template-based email generation."""
    first_name = name.split()[0] if name else "there"
    
    subject = f"Regarding {context[:50]}..." if len(context) > 50 else f"Regarding {context}"
    
    body = f"""Dear {first_name},

I hope this message finds you well!

I'm reaching out to you regarding the following:

{context}

I thought this would be particularly relevant for you and wanted to make sure you were informed.

If you have any questions or would like to discuss this further, please don't hesitate to reach out.

Looking forward to hearing from you.

Best regards,
Your Name

---
This email was sent to {email}
"""
    
    return subject, body

## Step 6: Email Sending Function

In [ ]:
def send_email(service, to_email, subject, body):
    """Send an email using Gmail API."""
    try:
        message = MIMEMultipart()
        message['to'] = to_email
        message['subject'] = subject
        
        msg_body = MIMEText(body, 'plain')
        message.attach(msg_body)
        
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')
        send_message = {'raw': raw_message}
        
        result = service.users().messages().send(userId='me', body=send_message).execute()
        
        return {
            'status': 'success',
            'message_id': result['id'],
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    except HttpError as error:
        return {
            'status': 'failed',
            'error': str(error),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    except Exception as e:
        return {
            'status': 'failed',
            'error': str(e),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

## Step 7: Main Email Agent Class

In [ ]:
class EmailAgent:
    """Main email agent class for personalized bulk email sending with Qwen2.5 0.5B."""
    
    def __init__(self):
        self.recipients = []
        self.context = ""
        self.generated_emails = []
        self.gmail_service = None
        self.use_llm = True
    
    def toggle_llm(self, use_llm=True):
        """Toggle between LLM and template-based generation."""
        self.use_llm = use_llm
        mode = "Qwen2.5 0.5B Instruct" if use_llm else "template-based"
        print(f"✅ Email generation mode: {mode}")
    
    def set_recipients(self, recipients_list):
        """
        Set recipients list.
        Expected format: [{'name': 'John Doe', 'email': 'john@example.com'}, ...]
        """
        self.recipients = recipients_list
        
        # Validate all emails
        invalid_emails = []
        for recipient in self.recipients:
            if not validate_email(recipient['email']):
                invalid_emails.append(recipient['email'])
        
        if invalid_emails:
            print(f"⚠️  WARNING: The following email addresses look invalid:")
            for email in invalid_emails:
                print(f"   - {email}")
            print()
        
        print(f"✅ {len(self.recipients)} recipients loaded.")
        return len(invalid_emails) == 0
    
    def set_context(self, context):
        """Set the email context/topic."""
        self.context = context
        print(f"✅ Context set: '{context[:100]}{'...' if len(context) > 100 else ''}'")
    
    def generate_emails(self):
        """Generate personalized emails for all recipients using Qwen2.5 or template."""
        self.generated_emails = []
        
        mode = "Qwen2.5 0.5B Instruct (FP16)" if self.use_llm else "template"
        print(f"\n🤖 Generating emails using {mode} mode...")
        
        if self.use_llm and qwen_generator.model is None:
            print("⚠️  Qwen2.5 model not loaded. Loading now...")
            if not qwen_generator.load_model():
                print("⚠️  Failed to load Qwen2.5. Switching to template mode.")
                self.use_llm = False
        
        for i, recipient in enumerate(self.recipients, 1):
            print(f"   [{i}/{len(self.recipients)}] Generating for {recipient['name']}...", end=' ')
            
            if self.use_llm:
                subject, body = generate_personalized_email_with_qwen(
                    recipient['name'],
                    recipient['email'],
                    self.context
                )
            else:
                subject, body = generate_personalized_email_template(
                    recipient['name'],
                    recipient['email'],
                    self.context
                )
            
            self.generated_emails.append({
                'name': recipient['name'],
                'email': recipient['email'],
                'subject': subject,
                'body': body
            })
            
            print("✅")
        
        print(f"\n✅ Generated {len(self.generated_emails)} personalized emails.")
    
    def display_emails(self):
        """Display all generated emails in a structured format."""
        if not self.generated_emails:
            print("❌ No emails generated yet. Run generate_emails() first.")
            return
        
        print("\n" + "="*80)
        print("GENERATED EMAILS - PREVIEW")
        print("="*80 + "\n")
        
        for i, email_data in enumerate(self.generated_emails, 1):
            print(f"📧 EMAIL #{i}")
            print(f"{'─'*80}")
            print(f"To: {email_data['name']} <{email_data['email']}>")
            print(f"Subject: {email_data['subject']}")
            print(f"\n{email_data['body']}")
            print(f"{'─'*80}\n")
        
        # Summary table
        table_data = [
            [i, email['name'], email['email'], email['subject'][:50] + '...' if len(email['subject']) > 50 else email['subject']]
            for i, email in enumerate(self.generated_emails, 1)
        ]
        
        print("\n📊 SUMMARY TABLE")
        print(tabulate(table_data, 
                      headers=['#', 'Name', 'Email', 'Subject'],
                      tablefmt='grid'))
        print()
    
    def send_emails(self):
        """Send all generated emails after confirmation."""
        if not self.generated_emails:
            print("❌ No emails to send. Generate emails first.")
            return
        
        # Authenticate with Gmail
        print("\n🔐 Authenticating with Gmail...")
        self.gmail_service = authenticate_gmail()
        
        if not self.gmail_service:
            print("❌ Authentication failed. Cannot send emails.")
            return
        
        print(f"\n📨 Sending {len(self.generated_emails)} emails...\n")
        
        results = []
        for i, email_data in enumerate(self.generated_emails, 1):
            print(f"Sending to {email_data['name']} ({email_data['email']})...", end=' ')
            
            result = send_email(
                self.gmail_service,
                email_data['email'],
                email_data['subject'],
                email_data['body']
            )
            
            result['name'] = email_data['name']
            result['email'] = email_data['email']
            results.append(result)
            
            if result['status'] == 'success':
                print("✅ Sent")
            else:
                print(f"❌ Failed: {result['error']}")
        
        # Final report
        print("\n" + "="*80)
        print("FINAL SENDING REPORT")
        print("="*80 + "\n")
        
        success_count = sum(1 for r in results if r['status'] == 'success')
        failed_count = len(results) - success_count
        
        print(f"✅ Successfully sent: {success_count}/{len(results)}")
        print(f"❌ Failed: {failed_count}/{len(results)}\n")
        
        # Detailed table
        table_data = [
            [
                i,
                r['name'],
                r['email'],
                '✅ Success' if r['status'] == 'success' else '❌ Failed',
                r['timestamp'],
                r.get('message_id', r.get('error', 'N/A'))[:30]
            ]
            for i, r in enumerate(results, 1)
        ]
        
        print(tabulate(table_data,
                      headers=['#', 'Name', 'Email', 'Status', 'Timestamp', 'Details'],
                      tablefmt='grid'))
        
        return results

# Initialize the agent
agent = EmailAgent()
print("✅ Email Agent initialized with Qwen2.5 0.5B Instruct (FP16) support!")

---
# 🚀 START HERE - INTERACTIVE SECTION

Now let's use the email agent! Follow the cells below to send personalized emails.

## A. Input Recipients and Context

**Edit the cell below** to add your recipients and email context.

In [ ]:
# ===== EDIT THIS SECTION =====

# List of recipients (name + email)
recipients = [
    {'name': 'Habibi Doe', 'email': 'kodlingepranav3@gmail.com'},
    {'name': 'Jane Smith', 'email': 'jane@example.com'},
]

# Email context/topic - BE DETAILED! Qwen2.5 will understand and use this context
email_context = """
Your detailed email context here...
Be as specific as possible - the AI will use all details!
"""

# ===== END EDIT SECTION =====

# Set recipients and context
agent.set_recipients(recipients)
agent.set_context(email_context)

## B. Generate Personalized Emails

Run this cell to generate all personalized emails.

In [ ]:
# Generate all emails
agent.generate_emails()

# Display all generated emails
agent.display_emails()

## C. Confirmation Before Sending

**IMPORTANT:** Review all emails above carefully before proceeding!

**Do you want to send these emails?**
- Edit the cell below and change `send_confirmation` to `"yes"` to send
- Keep it as `"no"` to skip sending

In [ ]:
# ===== EDIT THIS TO CONFIRM =====
send_confirmation = "yes"  # Change to "yes" to send emails
# ================================

if send_confirmation.lower() == "yes":
    print("✅ Confirmation received. Proceeding to send emails...\n")
    results = agent.send_emails()
else:
    print("❌ Sending cancelled. No emails were sent.")
    print("To send emails, change send_confirmation to 'yes' and run this cell again.")

---
## 📝 Usage Instructions

### First Time Setup:

1. **System Requirements:**
   - Python 3.8+
   - 4GB+ RAM (8GB recommended)
   - GPU optional (model works great on CPU too!)
   - ~1GB disk space for Qwen2.5 model

2. **Get Gmail API Credentials:**
   - Visit [Google Cloud Console](https://console.cloud.google.com/)
   - Create a new project
   - Enable Gmail API for your project
   - Create OAuth 2.0 credentials (select "Desktop app")
   - **Add yourself as a test user** in OAuth consent screen
   - Download credentials and save as `credentials.json` in this notebook's folder

3. **Install Dependencies:**
   - Run the first code cell to install required packages
   - This will take a few minutes

4. **Run Setup Cells:**
   - Execute cells in order from Step 2 through Step 7
   - **Cell 11 with `qwen_generator.load_model()`** will download Qwen2.5 0.5B (~1GB) on first run
   - This download happens only once

### Each Time You Want to Send Emails:

1. **Edit Input Cell (Section A):**
   - Add your recipients list with names and emails
   - **IMPORTANT**: Provide detailed context - Qwen2.5 will understand it fully!
   - Example: Instead of "product launch", write the full details about what you're launching, why it matters, pricing, timeline, benefits, etc.
   - The more context you provide, the better the personalized emails

2. **Generate Emails (Section B):**
   - Run the cell to generate Qwen2.5-powered personalized emails
   - Each email will be uniquely written by the AI for each recipient
   - **Super fast generation**: ~1-3 seconds per email on GPU, ~5-10 seconds on CPU
   - Review each email carefully

3. **Confirm and Send (Section C):**
   - If emails look good, change `send_confirmation` to `"yes"`
   - Run the cell to send
   - View the final sending report

### Features:
✅ **Qwen2.5 0.5B Instruct**: Ultra-fast lightweight model from Alibaba Cloud
✅ **FP16 Precision**: Optimized for both GPU and CPU with half-precision
✅ **Completely Local**: Runs on your machine - no API costs, no internet needed after download
✅ **AI-Written Emails**: Each email is uniquely written by the language model
✅ **Context-Aware**: Qwen2.5 understands complex context and generates natural emails
✅ **Blazing Fast**: Fastest option - 5-10x faster than larger models
✅ **Low Requirements**: Works great even on modest hardware
✅ **GPU Accelerated**: Uses CUDA if available, optimized for CPU too
✅ **Gmail Integration**: Uses your Gmail account via OAuth2
✅ **Email Validation**: Warns about invalid addresses
✅ **Preview & Confirm**: See all emails before sending
✅ **Privacy Friendly**: No data storage or external API calls
✅ **Fallback**: If generation fails, falls back to template mode

### Advanced Options:
- Disable LLM mode: `agent.toggle_llm(False)` for template-based emails
- Adjust generation temperature (in code): Lower = more focused, Higher = more creative

### Performance (with Qwen2.5 0.5B):
- **GPU (CUDA) with FP16**: ~1-3 seconds per email ⚡
- **CPU with FP16**: ~5-10 seconds per email
- **10x faster** than Phi-3 Mini, **20x faster** than Mistral 7B
- First generation includes model loading time (~5 seconds)